# Data Science

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fuyu-quant/langchain-tools/blob/main/examples/datascience.ipynb)

In [1]:
!pip install git+https://github.com/fuyu-quant/langchain-tools.git

  Cloning https://github.com/fuyu-quant/langchain-tools.git to /tmp/pip-req-build-245z6wvj
  Running command git clone --filter=blob:none --quiet https://github.com/fuyu-quant/langchain-tools.git /tmp/pip-req-build-245z6wvj
  Resolved https://github.com/fuyu-quant/langchain-tools.git to commit 90f8145b475870c9c78f774bdf04a64736ff823b
  Preparing metadata (setup.py) ... done
  Created wheel for langchaintools: filename=langchaintools-0.2.0-py3-none-any.whl size=9421 sha256=28b9248eb78f4cdb25504f2d4a766e05499c02c9565be0d8baf1bc2ac00e7404
  Stored in directory: /tmp/pip-ephem-wheel-cache-g7dynff2/wheels/ea/fc/e9/fd85c3556513c353cf0eaee43cd7473ffc93461ab46e639ae5
Successfully built langchaintools
  Attempting uninstall: langchaintools
    Found existing installation: langchaintools 0.1.1
    Uninstalling langchaintools-0.1.1:
      Successfully uninstalled langchaintools-0.1.1


In [2]:
%%capture
!pip install langchain==0.0.152
!pip install openai==00.27.5
!pip install lightgbm==3.3.5

In [5]:
# LangChain
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, Tool, tool


# langchaintools 
from langchaintools import mltools
from langchaintools import preprocessingtools as ppt



import os

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

In [6]:
tools = [mltools.LgbmTrainTool(), 
        mltools.LgbmInferenceTool(),
        ppt.OnehotEncodingTool(), 
        ppt.LabelEncodingTool(),
        ppt.TargetEncodingTool(),
        ppt.File0Tool(),
        ppt.FileMeansTool(),
        ppt.FileMedianTool(),
        ppt.MakeDatasetTool()
        ]

llm = OpenAI(temperature=0)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [7]:
target = 'target'

#input = "カラム名がSexのone-hotエンコーディングを行ってください。"
#input = "カラム名がEmbarkedのラベルエンコーディングを行ってください。"
#input = "カラム名がSex_femaleのターゲットエンコーディングを行ってください。"
#input = "カラム名がAgeの欠損値を0で埋めてください。"
#input = "train.csvとeval.csvを作成してください。"

#input = "LightGBMで学習してください。推論はしないでください"
input = "LightGBMで推論してください。"


prompt = """
{input_}一度toolを使ったら必ず終了してください．
目的変数は{target_}です．
""".format(input_=input, target_=target)


agent.run(prompt)



> Entering new AgentExecutor chain...
 LightGBMを使って推論する必要がある
Action: lgbm_inference_tool
Action Input: target

FileNotFoundError: [Errno 2] No such file or directory: 'trained_model.pkl'

In [7]:
query = "特徴量の名前を教えてください"
output = chat_tool_with_pandas_df(df_, query)



> Entering new AgentExecutor chain...

Thought: I need to find the name of the feature in the dataframe
Action: python_repl_ast
Action Input: df.columns
Observation: Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
Thought: I now know the names of the features
Final Answer: df.columns

> Finished chain.
特徴量の名前を教えてください


Please tell me the name of the feature.The operation must be applied directly to the original dataframe.The answer must be just python code
df.columns


In [8]:
output 

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [9]:
df_ = pd.read_csv('/home/langchain-tools/data/train.csv')
df_.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [10]:
df_encoded = pd.get_dummies(df_, columns=['Sex'])
df_encoded.head()

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_female,Sex_male
0,1,0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C,1,0
2,3,1,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S,1,0
4,5,0,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S,0,1


In [11]:
from langchain.agents import initialize_agent
from langchain.tools import BaseTool

In [47]:
class OnehotEncodingTool(BaseTool):
    name = "onehot_encoding_tool"
    description = """It is useful for onehot encoding when column names are given. The input should be a column name."""

    def _run(self, query: str) -> str:
        """Use the tool."""

        df = pd.read_csv('/home/langchain-tools/data/train2.csv')
        df = pd.get_dummies(df, columns=[query])
        df.to_csv('/home/langchain-tools/data/train2.csv', index=False)

        result = "one-hot encoding has been completed. " 
        return result

    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("BingSearchRun does not support async")

In [55]:
from sklearn.preprocessing import LabelEncoder

class LabelEncodingTool(BaseTool):
    name = "label_encoding_tool"
    description = """It is useful for label encoding when column names are given. The input should be a column name."""

    def _run(self, query: str) -> str:
        """Use the tool."""

        df = pd.read_csv('/home/langchain-tools/data/train2.csv')
        le = LabelEncoder()
        df[query] = le.fit_transform(df[query])
        df.to_csv('/home/langchain-tools/data/train2.csv', index=False)

        result = "label encoding has been completed. " 
        return result

    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("BingSearchRun does not support async")

In [56]:
from category_encoders import TargetEncoder

class TargetEncodingTool(BaseTool):
    name = "target_encoding_tool"
    description = """It is useful for target encoding when column names are given. The input receives the column names of the target column and the column name of the objective variable, separated by commas, such as 'A,B'."""

    def _run(self, query: str) -> str:
        """Use the tool."""
        a, b = query.split(",")

        df = pd.read_csv('/home/langchain-tools/data/train2.csv')
        te = TargetEncoder()
        df[a] = df[a].astype(str)
        df[a] = te.fit_transform(df[a], df[b])
        df.to_csv('/home/langchain-tools/data/train2.csv', index=False)

        result = "target encoding has been completed. " 
        return result

    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("BingSearchRun does not support async")

In [63]:
class File0Tool(BaseTool):
    name = "file_0_tool"
    description = """It is useful to fill in missing values with zeros when a column name is received. The input should be a column name."""

    def _run(self, query: str) -> str:
        """Use the tool."""

        df = pd.read_csv('/home/langchain-tools/data/train2.csv')
        df[query] = df[query].fillna(0)
        df.to_csv('/home/langchain-tools/data/train2.csv', index=False)

        result = "The missing values could be filled with 0." 
        return result

    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("BingSearchRun does not support async")

In [64]:
class FileMeansTool(BaseTool):
    name = "file_means_tool"
    description = """It is useful to fill in missing values with means when a column name is received. The input should be a column name."""

    def _run(self, query: str) -> str:
        """Use the tool."""

        df = pd.read_csv('/home/langchain-tools/data/train2.csv')
        df[query] = df[query].fillna(df[query].mean())
        df.to_csv('/home/langchain-tools/data/train2.csv', index=False)

        result = "The missing values could be filled with means." 
        return result

    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("BingSearchRun does not support async")

In [65]:
class FileMedianTool(BaseTool):
    name = "file_median_tool"
    description = """It is useful to fill in missing values with median when a column name is received. The input should be a column name."""

    def _run(self, query: str) -> str:
        """Use the tool."""

        df = pd.read_csv('/home/langchain-tools/data/sample2.csv')
        df[query] = df[query].fillna(df[query].median())
        df.to_csv('/home/langchain-tools/data/sample2.csv', index=False)

        result = "The missing values could be filled with median." 
        return result

    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("BingSearchRun does not support async")

In [71]:
from sklearn.model_selection import train_test_split

class MakeDatasetTool(BaseTool):
    name = "make_dataset_tool"
    description = """It is useful to create train.csv and eval.csv. The column name of the target variable is used as input."""

    def _run(self, query: str) -> str:
        """Use the tool."""

        df = pd.read_csv('/home/langchain-tools/data/sample2.csv')
        X = df.drop(query, axis=1)
        y = df[query]

        # 学習データと評価データに分割
        X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.25, random_state=3655)

        X_train[query] = y_train
        X_eval[query] = y_eval
        
        X_train.to_csv('/home/langchain-tools/data/train.csv', index=False)
        X_eval.to_csv('/home/langchain-tools/data/eval.csv', index=False)

        result = "train.csv and eval.csv have been created." 
        return result

    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("BingSearchRun does not support async")

In [72]:
tools = [OnehotEncodingTool(), LabelEncodingTool(),TargetEncodingTool(),
            File0Tool(),FileMeansTool(),FileMedianTool(),MakeDatasetTool()]

llm = OpenAI(temperature=0)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)



In [73]:
#input = "カラム名がSexのone-hotエンコーディングを行ってください。"
#input = "カラム名がEmbarkedのラベルエンコーディングを行ってください。"
#input = "カラム名がSex_femaleのターゲットエンコーディングを行ってください。目的変数はSurvivedです"
#input = "カラム名がAgeの欠損値を0で埋めてください。"
input = "train.csvとeval.csvを作成してください。目的変数はSurvivedです。"




prompt = """
{input_}一度toolを使ったら終了してください．
""".format(input_=input)


agent.run(prompt)



> Entering new AgentExecutor chain...
 I need to create train.csv and eval.csv with the target variable being Survived.
Action: make_dataset_tool
Action Input: Survived
Observation: train.csv and eval.csv have been created.
Thought: I now know the final answer
Final Answer: train.csv and eval.csv have been created with the target variable being Survived.

> Finished chain.


'train.csv and eval.csv have been created with the target variable being Survived.'

In [69]:
df = pd.read_csv('/home/langchain-tools/data/train2.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_female,Sex_male
0,1,0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,2,0.188908,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,0,0.742038,0
2,3,1,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,2,0.742038,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,2,0.742038,0
4,5,0,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,2,0.188908,1


In [35]:
te = TargetEncoder()
df['Sex_male'] = te.fit_transform(df['SibSp'], df['Survived'])
df.head()

,Unnamed: 0.1,Unnamed: 0,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_female,Sex_male
0,0,0,1,0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,2,0,1
1,1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,0,1,1
2,2,2,3,1,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,2,1,0
3,3,3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,2,1,1
4,4,4,5,0,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,2,0,0


In [2]:
import pickle
import pandas as pd
import numpy as np

import lightgbm as lgbm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, r2_score

from langchain.tools import BaseTool

In [3]:
class LgbmTrainTool(BaseTool):
    name = "lgbm_train_tool"
    description = """It is useful for learning LightGBM. The input is the column name of the target variable."""

    def _run(self, query: str) -> str:
      """Use the tool."""
      
      df = pd.read_csv('/home/langchain-tools/data/train.csv')
      x = df.drop([query], axis = 1)
      y = df[query]

      x_train,x_valid,y_train,y_valid = train_test_split(x, y ,test_size = 0.2, random_state=3655)

      # categorical features
      categorical_features = []
      for i in df.columns:
          if df[i].dtypes == 'category':
              categorical_features.append(i)


      lgb_train = lgbm.Dataset(x_train,y_train,categorical_feature=categorical_features,free_raw_data=False)
      lgb_eval = lgbm.Dataset(x_valid,y_valid,reference=lgb_train,categorical_feature=categorical_features,free_raw_data=False)


      # number of classes of the objective variable
      num_class = len(df[query].unique())
      if num_class == 2:
          params = {'task': 'train', 'boosting_type': 'gbdt','objective': 'binary', 'metric': 'auc', 'verbose': -1}
      elif num_class <= 50:
          params = {'task': 'train', 'boosting_type': 'gbdt','objective': 'multiclass', 'metric': 'multi_logloss','num_class': num_class, 'verbose': -1}
      else:
          params = {'task': 'train', 'boosting_type': 'gbdt','objective': 'regression','metric': 'rmse', 'verbose': -1}


      lgbm_model = lgbm.train(
          params,
          lgb_train,
          valid_sets=[lgb_train,lgb_eval],
          verbose_eval=10,
          #num_boost_round=1000,
          )
      

      file = '/home/langchain-tools/data/trained_model.pkl'
      pickle.dump(lgbm_model, open(file, 'wb'))

      split_df = pd.DataFrame(lgbm_model.feature_importance(importance_type = 'split'),index = x.columns, columns=['split_importance']).sort_values('split_importance', ascending=False)
      gain_df = pd.DataFrame(lgbm_model.feature_importance(importance_type = 'gain'),index = x.columns,columns=['gain_importance'])
      importance = split_df.join(gain_df)
      importance.to_csv('/home/langchain-tools/data/importance.csv')

      result = "LightGBM learning is complete."
      return result

    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("BingSearchRun does not support async")


class LgbmInferenceTool(BaseTool):
    name = "lgbm_inference_tool"
    description = """It is useful for inference using LightGBM. The input is the column name of the objective variable."""

    def _run(self, query: str) -> str:
        """Use the tool."""
        df = pd.read_csv('/home/langchain-tools/data/test.csv')


        file = 'trained_model.pkl'
        lgbm_model = pickle.load(open(file, 'rb'))

        df = pd.read_csv('/home/langchain-tools/data/test.csv')
        X = df.drop([query], axis = 1)
        y = df[query]

        num_class = len(df[query].unique())
        # binary classification
        if num_class == 2:
            y_pred = lgbm_model.predict(X, num_interation=lgbm_model.best_iteration)
            roc_auc = roc_auc_score(y, y_pred)
            y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]
            accuracy = accuracy_score(y, y_pred_binary)
            df_report = pd.DataFrame([roc_auc, accuracy], index=['roc_auc', 'accuracy'], columns=['score'])
            df_report.to_csv('/home/langchain-tools/data/report.csv')

        
        # multiclass classification
        elif num_class <= 50:
            y_pred = lgbm_model.predict(X, num_interation=lgbm_model.best_iteration)
            y_pred_class = [np.argmax(pred) for pred in y_pred]
            accuracy = accuracy_score(y, y_pred_class)
            #conf_mat = confusion_matrix(y, y_pred_class)
            df_report = pd.DataFrame([accuracy], index=['accuracy'], columns=['score'])
            df_report.to_csv('/home/langchain-tools/data/report.csv')


        
        # regression
        else:
            y_pred = lgbm_model.predict(X, num_interation=lgbm_model.best_iteration)
            mse = mean_squared_error(y, y_pred)
            r2 = r2_score(y, y_pred)
            df_report = pd.DataFrame([mse, r2], index=['mse', 'r2'], columns=['score'])
            df_report.to_csv('/home/langchain-tools/data/report.csv')




        result = "LightGBM inference is complete." 
        return result

    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("BingSearchRun does not support async")

In [4]:
tools = [LgbmTrainTool(), LgbmInferenceTool(),OnehotEncodingTool(), LabelEncodingTool(),TargetEncodingTool(),
            File0Tool(),FileMeansTool(),FileMedianTool(),MakeDatasetTool()]

llm = OpenAI(temperature=0)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

NameError: name 'OnehotEncodingTool' is not defined

In [115]:
#input = "LightGBMで学習してください。推論はしないでください"
input = "LightGBMで推論してください。"

target = 'target'

prompt = """
{input_}一度toolを使ったら必ず終了してください．
目的変数は{target_}です．
""".format(input_=input, target_=target)


agent.run(prompt)



> Entering new AgentExecutor chain...
 LightGBMを使って推論する必要がある
Action: lgbm_inference_tool
Action Input: target
Observation: LightGBM inference is complete.
Thought: 目的変数を推論したので終了
Final Answer: LightGBM inference is complete.

> Finished chain.


'LightGBM inference is complete.'